In [2]:
import os, sys

import matplotlib.pyplot as plt
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
import numpy as np

import numpy as np
from scipy.spatial import distance_matrix
from src.pso import Swarm
from src.graph import Graph

In [3]:
from IPython.core.display_functions import clear_output

import os
from scipy.spatial import distance_matrix
from src.graph import Graph


def read_tsp_file(file_path: str) -> np.ndarray:
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
        num_cities = int(lines[0])
        coords = []
        for line in lines[1:num_cities + 1]:
            x, y = map(float, line.split())
            coords.append([x, y])

        return np.array(coords)

In [ ]:
import matplotlib.pyplot as plt

coords = read_tsp_file('../data/tsp_51_1')
dist_mat = distance_matrix(coords, coords)
graph = Graph(dist_mat, coords, eval_nn=True)

swarm = Swarm(graph, particle_count=50, self_trust=0.4, past_trust=0.5, global_trust=0.7)
output = swarm.run(iterations=5000)
history = output.history



In [ ]:
plt.style.use('default')
plt.rc('font', family='serif')

plt.figure(figsize=(8, 5))
plt.plot(range(1, len(history) + 1), history, linewidth=1.5, label='PSO')
plt.xlabel('Итерация', fontsize=12)
plt.ylabel('Лучшая найденная длина пути', fontsize=12)
plt.legend(frameon=False)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('pso_convergence_tsp51.pdf', format='pdf', dpi=300)
plt.show()


In [4]:
import pandas as pd
import numpy as np
import concurrent
from concurrent.futures import ProcessPoolExecutor, as_completed
from collections import defaultdict


datasets = ['tsp_51_1', 'tsp_100_3', 'tsp_200_2']
n_runs = 10
iterations = 5000

def run_pso(ds_name: str):
    coords = read_tsp_file(f'../data/{ds_name}')
    dist_mat = distance_matrix(coords, coords)
    graph = Graph(dist_mat, coords, eval_nn=True)

    pso = Swarm(graph, particle_count=20, self_trust=0.4, past_trust=0.2, global_trust=0.2)
    output = pso.run(iterations=iterations)

    return output.best_dist, output.overall_time

def wrapper(args):
    ds = args
    bd, tm = run_pso(ds)
    return ds, bd, tm

if __name__ == '__main__':
    tasks = [ds for ds in datasets for _ in range(n_runs)]
    total_tasks = len(tasks)
    
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(wrapper, task) for task in tasks]
        
        results = []
        completed = 0
        print("Прогресс выполнения:")
        for future in as_completed(futures):
            results.append(future.result())
            completed += 1
            progress = (completed / total_tasks) * 100
            print(f"\r{progress:.1f}% ({completed}/{total_tasks})", end="", flush=True)

    grouped = defaultdict(lambda: {'bests': [], 'times': []})
    for ds, bd, tm in results:
        grouped[ds]['bests'].append(bd)
        grouped[ds]['times'].append(tm)

    records = []
    for ds in datasets:
        data = grouped[ds]
        records.append({
            'Dataset': ds,
            'Mean Best Distance': np.mean(data['bests']),
            'Min Best Distance': np.min(data['bests']),
            'Mean Time (s)': np.mean(data['times']),
        })

    df = pd.DataFrame(records)
    print("\n\nРезультаты:")
    print(df)

    with open('pso_summary.tex', 'w') as f:
        f.write(df.to_latex(
            index=False,
            float_format="%.2f",
            caption="Сравнительная статистика обычного PSO по трём TSP‑наборaм (5×5000 итераций)",
            label="tab:pso_summary",
        ))

Прогресс выполнения:
100.0% (30/30)

Результаты:
     Dataset  Mean Best Distance  Min Best Distance  Mean Time (s)
0   tsp_51_1          465.018083         459.785586    1662.154330
1  tsp_100_3        23192.462128       23164.502392    3198.352998
2  tsp_200_2        35252.774195       34855.646066    5144.906576
